# Factorisation LU

Mettez ci-dessous les imports classiques de librairie Python

In [1]:
%matplotlib inline
import numpy as np
import numpy.linalg as npl
import matplotlib.pyplot as plt

M = np.load('TP6_Matrice.npy')

Le but de ce TP est d'implémenter la factorisation LU, avec et sans permutations, et de tester ces méthodes pour résoudre un système linéaire. Dans cette dernière partie, on implémentera un algorithme de résolution de systèmes triangulaires.

## 1) Algorithme de factorisation LU standard

On se propose tout d'abord d'implémenter un code de factorisation LU qui prendra en entrée une matrice $A$ carré de taille quelconque et qui donnera en sortie le couple de matrice L et U correspondant à la factorisation LU de la matrice A. On rappelle que la factorisation LU s'obtient à partir de la méthode de pivot de Gauss. Plus précisément, la factorisation LU peut se résumer à l'algorithme suivant

$$
A^{(0)} = A\in M_n(\mathbb{R}),\\
L^{(0)} = 0 \in M_n(\mathbb{R}),\\
\textrm{Pour}\; k = 0,1,2,\cdots,n-1:\;
\left\{\begin{array}{ll}
\ell^{(k)} = \left\{\begin{array}{ll}
0,\;\textrm{pour}\;1\leq i\leq k-1,\\
\frac{a_{i,k}^{(k)}}{a_{k,k}^{(k)}},\;\textrm{pour}\;k\leq i\leq n,
\end{array}\right.\\
L^{(k+1)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_i,\; \textrm{pour}\; j = k\;\textrm{et}\; k\leq i\leq n,
\\ L^{(k)}_{i,j}\; \textrm{sinon},
\end{array}\right.\\
B^{(k)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_{i} a^{(k)}_{k,j},\; \textrm{pour}\; k+1\leq i\leq n \;\textrm{et}\; k\leq j\leq n,
\\ 0\; \textrm{sinon},
\end{array}\right.,\\
A^{(k+1)} = A^{(k)} - B^{(k)},\\
\end{array}\right.
$$

où
- $\ell^{(k)}$ est un vecteur de $\mathbb{R}^n$,
- $A^{(k)}$ est une matrice de $M_n(\mathbb{R})$ dont on note les coefficients $(a^{(k)}_{i,j})_{1\leq i,j\leq n}$.

A la fin de cette algorithme, on obtient $L = L^{(n-1)}$ et $U = A^{(n-1)}$.

> **A faire :** Implémenter la fonction `LU` qui permet de calculer la factorisation LU d'une matrice $A$ carré de dimension quelconque à partir de l'algorithme précédent. On prendra soin de vérifier si le pivot est non-nul et, dans le cas contraire, on renverra une erreur à l'utilisateur. On pourra tester la fonction sur la matrice
$$
A = \left(\begin{array}{ccc}
2 & 1 & 1 \\
6 & 2 & 1 \\
-2 & 2 & 1
\end{array}\right),
$$
dont la décomposition LU est donnée par
$$
L = \left(\begin{array}{ccc}
1 & 0 & 0 \\
3 & 1 & 0 \\
-1 & -3 & 1
\end{array}\right)\quad\textrm{et}\quad U = \left(\begin{array}{ccc}
2 & 1 & 1 \\
0 & -1 & -2 \\
0 & 0 & -4
\end{array}\right).
$$

In [3]:
"""
AUTRE METHODE : Marche pas avec toutes les matrices
def LU(A):
    U = np.copy(A)
    L = np.identity(len(A),dtype=int)
    for i in range(len(U)) : 
        for j in range(i+1, len(U)):
            k = U[j][i]/U[i][i]
            U[j]=U[j]-k*U[i]
            L[j][i]=k
    return L,U
"""

def LU(A):
    n=len(A)
    L=np.zeros((n,n))
    U=np.copy(A)
    for k in range(0,n):
        if U[k][k]==0:
            raise Exception("PivotNul")
        l=np.zeros(n)
        #for i in range(k,n):
        #    l[i]=U[i][k]/U[k][k]
        l[k:]=U[k:,k]/U[k,k]
        L[k:,k] = l[k:]
        B=np.zeros((n,n))
        for p in range(k,n):
            for i2 in range(k+1,n):
                B[i2][p]=l[i2]*U[k][p]
        U=U-B
    return L,U

In [4]:
A = np.array([[2,1,1],[6,2,1],[-2,2,1]])
#Donner la valeurs des arguments de retour de fonctions
L,U = LU(A)
print("L=",L)
print("U=",U)
#print(A-L.dot(U))

L= [[ 1.  0.  0.]
 [ 3.  1.  0.]
 [-1. -3.  1.]]
U= [[ 2.  1.  1.]
 [ 0. -1. -2.]
 [ 0.  0. -4.]]


## 2) Algorithme de factorisation LU avec stratégie de pivot partiel

On passe maintenant à la factorisation LU avec la stratégie de pivot partiel. Cette méthode met en place, durant le pivot de Gauss, des permutations sur les lignes qui permettent de diminuer les erreurs numériques et de parer au problème d'un pivot nul. Ces permutations sur les lignes vont faire en sorte que le pivot dans la méthode de Gauss est toujours le plus grand possible. En pratique, à chaque étape $k$, lors du choix du pivot, on va chercher dans la colonne $k$ le plus grand pivot (en valeur absolue) puis on va procèder à une permutation entre la ligne de ce pivot et la ligne $k$. Plus précisément, la factorisation LU avec permutations peut se résumer à l'algorithme suivant

$$
A^{(0)} = A\in M_n(\mathbb{R}),\\
L^{(0)} = 0\in M_n(\mathbb{R}),\\
P^{(0)} = \textrm{Id}\in M_n(\mathbb{R}),\\
\textrm{Pour}\; k = 0,1,2,\cdots,n-1:\;
\left\{\begin{array}{ll}
m = \textrm{argmax}_{k\leq j \leq n} |a_{j,k}^{(k)}|\\
Q^{k,m} = \textrm{Matrice de permutation des indices $k$ et $m$}\\
P^{(k+1)} = Q^{k,m}P^{(k)},\\
L^{(k+1/2)} = Q^{k,m}L^{(k)},\\
A^{(k+1/2)} = Q^{k,m}A^{(k)},\\
\ell^{(k)} = \left\{\begin{array}{ll}
0,\;\textrm{pour}\;1\leq i\leq k-1,\\
\frac{a_{i,k}^{(k+1/2)}}{a_{k,k}^{(k+1/2)}},\;\textrm{pour}\;k\leq i\leq n,
\end{array}\right.\\
L^{(k+1)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_i,\; \textrm{pour}\; j = k\;\textrm{et}\; k\leq i\leq n,
\\ L^{(k+1/2)}_{i,j}\; \textrm{sinon},
\end{array}\right.\\
B^{(k)}_{i,j} = \left\{\begin{array}{ll}
\ell^{(k)}_{i} a^{(k+1/2)}_{k,j},\; \textrm{pour}\; k+1\leq i\leq n \;\textrm{et}\; k\leq j\leq n,
\\ 0\; \textrm{sinon},
\end{array}\right.,\\
A^{(k+1)} = A^{(k+1/2)} - B^{(k)},\\
\end{array}\right.
$$

A la fin de cette algorithme, on obtient $L = L^{(n-1)}$, $U = A^{(n-1)}$ et $P = P^{(n-1)}$. La matrice de permutation n'a pas vraiment besoin d'être assemblée, on pourra simplement écrire une fonction `Permut` qui prendra en argument une matrice et le couple d'indice et donnera en sortie la matrice avec ses lignes permutées. Afin d'identifier l'indice du plus grand pivot, on pourra s'aider de la fonction `argmax`. On rappelle de plus que, pour une matrice de permutation $P$, on a

$$
P^{-1} = P^T.
$$

> **A faire :** Implémenter la fonction `PLU` qui permet de calculer la factorisation LU d'une matrice $A$ carré de dimension quelconque à partir de l'algorithme précédent. On prendra soin de vérifier si le pivot est non-nul et, dans le cas contraire, on renverra une erreur à l'utilisateur. On pourra tester la fonction sur la matrice
$$
A = \left(\begin{array}{ccc}
1 & 1 & 1 & 1 \\
1 & 1 & 3 & 3 \\
1 & 1 & 2 & 3\\
1 & 3 & 3 & 3
\end{array}\right),
$$
dont la décomposition PLU est donnée par
$$
P = \left(\begin{array}{ccc}
1 & 0 & 0 & 0\\
0 & 0 & 0 & 1\\
0 & 1 & 0 & 0\\
0 & 0 & 1 & 0
\end{array}\right),\quad L = \left(\begin{array}{ccc}
1 & 0 & 0 & 0\\
1 & 1 & 0 & 0\\
1 & 0 & 1 & 0\\
1 & 0 & 1/2 & 1
\end{array}\right)\quad\textrm{et}\quad U = \left(\begin{array}{ccc}
1 & 1 & 1 & 1\\
0 & 2 & 2 & 2\\
0 & 0 & 2 & 2\\
0 & 0 & 0 & 1
\end{array}\right).
$$

In [5]:
#echange les lignes
def Permut(A,i,j):
#prend tout la colonne k et toutes les lignes à partir de la k ième:(tout dans le même crochet)
#verification lu: PA-LU=0 si c'est ok 
#Produit matrciel: utiliser np.dot
    n=len(A)
    B = np.copy(A)
    B[i] = A[j].copy()
    B[j] = A[i].copy()
    return B
    
def PLU(A):
    n=len(A)
    L=np.zeros((n,n))
    l=np.zeros(n)
    B=np.zeros((n,n))
    U=np.zeros((n,n))
    P=np.identity(n)
    m=0
    for k in range(0,n):
        m=k+np.argmax(abs(A[k:,k]))
        A=Permut(A,k,m)
        L=Permut(L,k,m)
        P=Permut(P,k,m)
        for j in range(k-1,n):
            for i in range(k,n):
                if A[k][k]==0:
                    raise Exception("PivotNul")
                if j==k:
                    L[i][j]=l[i]
                l[i]=A[i][k]/A[k][k]
                for p in range(k,n):
                    for i2 in range(k+1,n):
                        B[i2][p]=l[i2]*A[k][p]
            A=A-B
    U=A
    return P,L,U

def PLU(A):
    n=len(A)
    L=np.zeros((n,n))
    U=np.copy(A)
    P=np.identity(n)
    for k in range(0,n):
        m=k+np.argmax(abs(U[k:,k]))
        U=Permut(U,k,m)
        L=Permut(L,k,m)
        P=Permut(P,k,m)
        if U[k][k]==0:
            raise Exception("PivotNul")
        l=np.zeros(n)
        #for i in range(k,n):
        #    l[i]=U[i][k]/U[k][k]
        l[k:]=U[k:,k]/U[k,k]
        L[k:,k] = l[k:]
        B=np.zeros((n,n))
        for p in range(k,n):
            for i2 in range(k+1,n):
                B[i2][p]=l[i2]*U[k][p]
        U=U-B
    return P,L,U

In [6]:
A = np.array([[1,1,1,1],[1,1,3,3],[1,1,2,3],[1,3,3,3]])
P,L,U = PLU(A)
print("P=",P)
print("L=",L)
print("U=",U)
print(np.dot(P,A)-np.dot(L,U)) #pour vérification
print(A)

P= [[1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
L= [[1.  0.  0.  0. ]
 [1.  1.  0.  0. ]
 [1.  0.  1.  0. ]
 [1.  0.  0.5 1. ]]
U= [[1. 1. 1. 1.]
 [0. 2. 2. 2.]
 [0. 0. 2. 2.]
 [0. 0. 0. 1.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[1 1 1 1]
 [1 1 3 3]
 [1 1 2 3]
 [1 3 3 3]]


## 3) Résolution de systèmes triangulaires et comparaison des méthodes

Nous allons à présent comparer les 2 méthodes que nous venons d'implémenter. Le but est de tester ces méthodes sur un système linéaire. Or, on sait que pour résoudre un tel système il faut passer par la résolution de systèmes triangulaires (supérieurs et inférieurs). Ainsi, avant de passer à la suite, on se propose d'implémenter une méthode permettant de résoudre un système triangulaire, qu'il soit inférieur ou supérieur.

On rappelle que l'algorithme de résolution pour un système triangulaire de la forme

$$
Tx = b,
$$

s'écrit, si $T$ est triangulaire inférieure,

$$
x_1 = b_1/T_{1,1}\\
\textrm{Pour}\; k = 2,3,\cdots, n\; :
x _k = \left(b_k - \sum_{j = 1}^{k-1} T_{k,j} x_j\right)/T_{k,k},
$$

et, si $T$ est triangulaire supérieure,

$$
x_n = b_n/T_{n,n}\\
\textrm{Pour}\; k = n-1,n-2,\cdots, 1\; :
x _k = \left(b_k - \sum_{j = k+1}^{n} T_{k,j} x_j\right)/T_{k,k}.
$$

> **A faire :** Implémenter la fonction `Solve_Triang` qui permet de résoudre un système linéaire triangulaire (inférieur ou supérieur). Cette fonction prendra en entrée une matrice et un vecteur (correspondant aux données du système linéaire) ainsi qu'une variable précisant si le système est triangulaire supérieur ou inférieur et donnera en sortie un vecteur qui est la solution que l'on cherche.

In [7]:
def Solve_Triang(T,b,s = 0):
    n=len(T)
    x=np.zeros(n)
    if s==0:
        x[0]=b[0]/T[0][0]
        for k in range(1,n):
            somme=0
            for j in range(0,k):
                somme=somme+T[k][j]*x[j]
            x[k]=(b[k]-somme)/T[k][k]
    else:
        for k in range(n-1,-1,-1):
            somme=0
            for j in range(n-1,k,-1):
                somme=somme+T[k][j]*x[j]
            x[k]=(b[k]-somme)/T[k][k]
    return x


On a charger, en début de ce notebook, la matrice $M$ de taille 100. A l'aide des deux méthodes de factorisation et de la méthode de résolution d'un système triangulaire, vous allez résoudre le système linéaire

$$
Mx = b,
$$

où $b\in\mathbb{R}^{100}$ est tel que $b_i = 1$, pour tout $1\leq i\leq n$.

> **A faire :** Résoudre le système précédent à l'aide de la factorisation LU standard et celle avec permutations. Comparer les résultats obtenus avec la solution obtenue avec `npl.solve(M,b)`. Quelle méthode donne le meilleur résultat?

In [8]:
b = np.ones(100)
L,U=LU(M)
y=Solve_Triang(L,b)
x=Solve_Triang(U,y,1)
print(x)
P,L1,U1=PLU(M)
y1=Solve_Triang(L1,P.dot(b))
x1=Solve_Triang(U1,y,1)
print(x1)
print(npl.solve(M,b))
#PLU est plus efficace que LU

[ 1.26709391e-02  7.92944712e-03  1.36956084e-02  7.72348653e-03
 -1.36187698e-02 -6.49516613e-03  2.18949164e-03  5.23534824e-04
  8.32724079e-03  4.69173983e-03 -1.35230632e-03  1.06748543e-02
  8.20833915e-03 -2.46004170e-03  7.33792724e-03  3.97266892e-03
  1.30969941e-03  4.68042590e-03 -6.31025062e-03  7.56354613e-04
  2.73711753e-03 -1.34304910e-05  4.58420441e-03  1.15648657e-03
  4.77128392e-03  9.64477290e-04 -2.64923224e-04 -8.47487391e-03
 -1.61838492e-02 -1.68434262e-02  3.17354401e-03  5.53955753e-03
  3.74308209e-03 -3.21467109e-03 -4.03371293e-03  7.90209841e-04
  3.21307545e-03  7.46660256e-03 -4.86634536e-03 -6.79164824e-03
  3.01622380e-03 -3.17423586e-03  5.53100448e-03  1.30955230e-02
  7.45641395e-05 -2.79039148e-03  8.13220100e-03  3.37058812e-03
 -1.77546624e-03 -6.78222235e-03 -5.65585840e-03 -4.45914194e-03
 -1.03719223e-03 -9.22282549e-04 -5.62829031e-04 -8.07555165e-03
 -5.71828948e-03  2.36033459e-03  2.42439076e-04  4.81056900e-03
 -4.52840858e-04  3.91658

In [9]:
A = np.array([[1,0],[1,2]])
b =np.ones(2)
x = npl.solve(A,b)
xtr = Solve_Triang(A,b,0)
print(x,xtr)

[1. 0.] [1. 0.]
